Start multiple instances of `iperf3` in parallel :

```bash
docker run -t --rm badouralix/toolbox parallel --ungroup --jobs 2 iperf3 -s -p {} ::: 8080 8081
```

More info on this command in <https://www.baeldung.com/linux/processing-commands-in-parallel#using-gnu-parallel>.

## Setup

The goal is to implement the following tree.

```mermaid
graph LR
    A("kernel")
    B(("1:0\nroot"))
    A <-- "if dport == 8080 -> 1:80\nif dport == 8081 -> 1:81" --> B
    B  --> C(("1:1\nclass:\n100mbits (htb)"))
    C  --> D(("1:80\nclass:\n80mbits (htb)"))
    C  --> E(("1:81\nclass:\n80mbits (htb)"))
```

In [1]:
tc qdisc add dev eth0 root handle 1:0 htb default 30
tc qdisc show dev eth0

qdisc htb 1: root refcnt 2 r2q 10 default 0x30 direct_packets_stat 0 direct_qlen 1000


In [2]:
tc class add dev eth0 parent 1:0 classid 1:1 htb rate 100mbit
tc class add dev eth0 parent 1:1 classid 1:80 htb rate 5mbit ceil 80mbit
tc class add dev eth0 parent 1:1 classid 1:81 htb rate 5mbit ceil 80mbit
tc class show dev eth0

class htb 1:80 parent 1:1 prio 0 rate 5Mbit ceil 80Mbit burst 1600b cburst 1600b 
class htb 1:81 parent 1:1 prio 0 rate 5Mbit ceil 80Mbit burst 1600b cburst 1600b 
class htb 1:1 root rate 100Mbit ceil 100Mbit burst 1600b cburst 1600b 


We want to match the traffic going to `dst 172.16.0.0/12` : `match ip dst 172.16.0.0/12`.
And we also want to match the traffic going to port `8080` : `match ip dport 8080 0xffff`. Here the destination port is `8080` and the mask is `0xffff` meaning only one port. See <https://serverfault.com/a/231894>.

More info on this in <https://tldp.org/HOWTO/Adv-Routing-HOWTO/lartc.qdisc.filters.html>.

In [3]:
tc filter add dev eth0 protocol ip parent 1:0 prio 1 u32 match ip dst 172.16.0.0/12 match ip dport 8080 0xffff flowid 1:80
tc filter add dev eth0 protocol ip parent 1:0 prio 1 u32 match ip dst 172.16.0.0/12 match ip dport 8081 0xffff flowid 1:81
tc filter show dev eth0

filter parent 1: protocol ip pref 1 u32 chain 0 
filter parent 1: protocol ip pref 1 u32 chain 0 fh 800: ht divisor 1 
filter parent 1: protocol ip pref 1 u32 chain 0 fh 800::800 order 2048 key ht 800 bkt 0 flowid 1:80 not_in_hw 
  match ac100000/fff00000 at 16
  match 00001f90/0000ffff at 20
filter parent 1: protocol ip pref 1 u32 chain 0 fh 800::801 order 2049 key ht 800 bkt 0 flowid 1:81 not_in_hw 
  match ac100000/fff00000 at 16
  match 00001f91/0000ffff at 20


## Experiments

### Single client

Here below we validate that the initial setup is correct and that both port `8080` and port `8081` are limited to 80Mbits/sec.

In [4]:
iperf3 -c 172.17.0.2 -p 8080 -t 10

Connecting to host 172.17.0.2, port 8080
[  5] local 172.17.0.3 port 51474 connected to 172.17.0.2 port 8080
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  12.4 MBytes   104 Mbits/sec    0   1.50 MBytes       
[  5]   1.00-2.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.50 MBytes       
[  5]   2.00-3.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.50 MBytes       
[  5]   3.00-4.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.50 MBytes       
[  5]   4.00-5.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.50 MBytes       
[  5]   5.00-6.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.50 MBytes       
[  5]   6.00-7.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.50 MBytes       
[  5]   7.00-8.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.50 MBytes       
[  5]   8.00-9.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.50 MBytes       
[  5]   9.00-10.00  sec  10.0 MBytes  83.9 Mbits/sec    0   1.50 MBytes       
- - - - - - - - - - - - - - - - - - 

In [5]:
iperf3 -c 172.17.0.2 -p 8081 -t 10

Connecting to host 172.17.0.2, port 8081
[  5] local 172.17.0.3 port 38812 connected to 172.17.0.2 port 8081
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  12.3 MBytes   103 Mbits/sec    0   1.37 MBytes       
[  5]   1.00-2.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.37 MBytes       
[  5]   2.00-3.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.37 MBytes       
[  5]   3.00-4.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.37 MBytes       
[  5]   4.00-5.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.37 MBytes       
[  5]   5.00-6.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.37 MBytes       
[  5]   6.00-7.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.37 MBytes       
[  5]   7.00-8.00   sec  8.75 MBytes  73.4 Mbits/sec    0   1.37 MBytes       
[  5]   8.00-9.00   sec  10.0 MBytes  83.9 Mbits/sec    0   1.37 MBytes       
[  5]   9.00-10.00  sec  8.75 MBytes  73.4 Mbits/sec    0   1.37 MBytes       
- - - - - - - - - - - - - - - - - - 

### Parallel clients

Now we run two clients in parallel, and we expect the total bandwidth to be limited to 100Mbits/sec as per the rate on the class `1:1`.

In [6]:
parallel --tagstring "port {} |" --jobs 2 iperf3 -c 172.17.0.2 -p {} -t 10 ::: 8080 8081

port 8080 |	Connecting to host 172.17.0.2, port 8080
port 8080 |	[  5] local 172.17.0.3 port 40310 connected to 172.17.0.2 port 8080
port 8080 |	[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
port 8080 |	[  5]   0.00-1.00   sec  8.69 MBytes  72.8 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   1.00-2.00   sec  6.25 MBytes  52.5 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   2.00-3.00   sec  5.00 MBytes  41.9 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   3.00-4.00   sec  6.25 MBytes  52.4 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   4.00-5.00   sec  6.25 MBytes  52.4 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   5.00-6.00   sec  5.00 MBytes  41.9 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   6.00-7.00   sec  6.25 MBytes  52.4 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   7.00-8.00   sec  5.00 MBytes  41.9 Mbits/sec    0   1.37 MBytes       
port 8080 |	[  5]   8.00-9.00   sec  6.25 MBytes  52.4 Mbits/s

## Cleanup

In [7]:
tc qdisc del dev eth0 root
tc qdisc show dev eth0

qdisc noqueue 0: root refcnt 2 
